In [1]:
!pip install -qU polars catboost optuna

In [2]:
import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs

import matplotlib.pyplot as plt
import seaborn as sns 

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, classification_report

from catboost import CatBoostClassifier, Pool, EShapCalcType, EFeaturesSelectionAlgorithm

import optuna

## 0. Dataset load and train test split

In [3]:
DATA_PATH = '/kaggle/input/sber-salaraclient-churn-data/'

In [4]:
data = pl.read_parquet(DATA_PATH + 'data_cleaned.parquet')
data.head()

target,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature12,feature14,feature16,feature17,feature18,feature19,feature22,feature24,feature25,feature31,feature33,feature35,feature36,feature37,feature38,feature41,feature43,feature44,feature45,feature46,feature47,feature48,feature49,feature52,feature53,feature54,feature55,…,feature992,feature993,feature994,feature996,feature997,feature998,feature999,feature1000,feature1001,feature1002,feature1003,feature1004,feature1035,feature1036,feature1037,feature1038,feature1042,feature1043,feature1045,feature1049,feature1050,feature1051,feature1052,feature1053,feature1054,feature1055,feature1056,feature1057,feature1059,feature1063,feature1064,feature1065,feature1066,feature1067,feature1068,feature1069,feature1076
i8,i16,i16,i16,i32,i16,i16,i8,i16,i16,i16,i16,i16,i16,i8,i16,i32,i16,i16,i16,i32,i32,i16,i32,i16,i16,i32,i16,i16,i32,i32,i16,i16,i16,i16,i32,i32,…,i16,i16,i16,i8,i8,i8,i8,i8,i8,i16,i16,i16,i32,i32,i16,i32,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i16,i32,i32,i32,i32,i32,i32,i32,i32,i32
0,1761,1759,85,105469,191,46,0,6,95,20,1,50,60,7,248,91601,173,1722,633,50524,16130,3652,85495,14172,7705,19131,6511,12018,31287,27091,0,0,0,0,30270,27157,…,5,8,16,1,0,0,2,1,2,3,6,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12045,12107,12510,21126,28913,48985,84264,84264
0,1761,1759,120,105610,144,71,0,135,135,187,1,43,101,11,18,36890,184,1722,633,50524,18999,3652,85495,14172,7705,59900,6511,12018,116284,141739,0,0,0,0,112320,138924,…,4,10,22,0,0,0,3,0,2,3,9,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,890,1759,141,105227,191,11,0,0,0,187,1,0,0,28,248,140184,248,1722,633,50524,65748,3652,85495,14172,7705,11110,6511,12018,261217,284839,0,0,0,0,252909,278364,…,9,24,40,1,1,2,3,3,5,8,23,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33735,31242,12691,48682,55949,70006,113317,113317
0,1599,966,30,102441,191,8,0,0,0,187,1,0,0,28,248,140184,248,1722,633,50524,65748,3652,85495,81,7705,38606,6511,12018,303119,151593,0,0,0,0,293748,148572,…,5,5,5,1,1,1,2,2,3,3,3,3,12436,0,0,12329,101,0,114,0,0,0,0,0,0,0,5220,0,5810,0,0,0,0,0,6872,8530,8530
0,1761,1759,85,104006,191,2,0,0,46,2,1,1,21,3,248,140184,248,1722,633,16988,65748,3652,0,14172,7705,17927,6511,12018,245560,449478,0,0,0,0,237725,438447,…,6,12,14,1,1,0,3,1,3,4,10,12,27032,0,0,26719,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7642,7642


In [5]:
print('Data shape and size:', data.shape, round(data.estimated_size('gb'), 3))
print('Target info:', pl.Series(data.select(pl.col('target'))).value_counts())

Data shape and size: (519615, 641) 0.659
Target info: shape: (2, 2)
┌────────┬────────┐
│ target ┆ count  │
│ ---    ┆ ---    │
│ i8     ┆ u32    │
╞════════╪════════╡
│ 0      ┆ 501078 │
│ 1      ┆ 18537  │
└────────┴────────┘


In [6]:
data.n_unique()

519615

In [7]:
n_unique_vals_by_col = data.drop(
    'target'
).select(
    pl.all().n_unique()
).transpose(
    include_header=True
).rename(
    {'column_0': 'n_unique'}
).sort(
    'n_unique'
)
n_unique_vals_by_col

column,n_unique
str,u32
"""feature242""",2
"""feature384""",2
"""feature385""",2
"""feature386""",2
"""feature387""",2
…,…
"""feature136""",501444
"""feature142""",501904
"""feature206""",502242


In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(
    data.drop('target').to_pandas(), data['target'].to_pandas().values, test_size=0.25, random_state=42, stratify=data['target'].to_pandas()
)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((389711, 640), (129904, 640), (389711,), (129904,))

## 1. Parameter tuning

In [9]:
default_params = {
    'random_state': 42, 'allow_writing_files': False, 'verbose': 1000, 'task_type': 'GPU',
}

def objective(trial):
    params = {
        'undersampling': trial.suggest_categorical('undersampling', [True, False]),
        'n_unique_to_cat_cols': trial.suggest_int('n_unique_to_cat_cols', 0, 25, step=5),
        # boosting params
        'iterations': trial.suggest_int('iterations', 500, 2500, step=500),
        'depth': trial.suggest_int('depth', 4, 8, step=1), # 6-10 catboost recoms
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 5, 25, step=2),
        'border_count': trial.suggest_int('border_count', 64, 128, step=64),
        'max_ctr_complexity': trial.suggest_int('max_ctr_complexity', 0, 7, step=1),
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli']),
    }
    if params['bootstrap_type'] == 'Bayesian':
        params['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0, 1)
    elif params['bootstrap_type'] == 'Bernoulli':
        params['subsample'] = trial.suggest_float('subsample', 0.5, 1.0)

    sampled_train = np.random.choice(len(X_train), size=int(len(X_train) * 0.9), replace=False)

    X_train_sampled, y_train_sampled = X_train.iloc[sampled_train].reset_index(drop=True), y_train[sampled_train]

    if params.pop('undersampling'):
        X_train_sampled['y'] = y_train_sampled
        Xy_train_sampled = X_train_sampled.groupby('y').head(100_000)

        X_train_sampled, y_train_sampled = Xy_train_sampled.drop(columns=['y']), Xy_train_sampled['y'].values

    n_unique_to_cat_cols = params.pop('n_unique_to_cat_cols')

    cat_cols = n_unique_vals_by_col.filter(
        pl.col('n_unique') <= n_unique_to_cat_cols
    )['column'].to_list()

    train_pool = Pool(X_train_sampled, label=y_train_sampled, cat_features=cat_cols)

    model = CatBoostClassifier(**params, **default_params)
    model.fit(train_pool)

    roc_auc_score_score = roc_auc_score(y_valid, model.predict_proba(X_valid)[:, 1])

    return roc_auc_score_score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, n_jobs=1)

print(f'Number of finished trials: {len(study.trials)}')

print(f'Best trial:')
trial = study.best_trial

print(f'  Value: {trial.value}')

print(f'  Params:')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

[I 2024-03-13 18:51:52,472] A new study created in memory with name: no-name-56a8fa76-3f46-422b-aec4-0e8ba0712dab


0:	learn: 0.6668199	total: 8.5s	remaining: 5h 54m 2s
1000:	learn: 0.2752140	total: 32.5s	remaining: 48.7s
2000:	learn: 0.2493313	total: 56.7s	remaining: 14.1s
2499:	learn: 0.2374208	total: 1m 8s	remaining: 0us


[I 2024-03-13 18:53:39,763] Trial 0 finished with value: 0.7621774084938122 and parameters: {'undersampling': True, 'n_unique_to_cat_cols': 25, 'iterations': 2500, 'depth': 7, 'l2_leaf_reg': 17, 'border_count': 128, 'max_ctr_complexity': 1, 'bootstrap_type': 'Bernoulli', 'subsample': 0.8267422253593207}. Best is trial 0 with value: 0.7621774084938122.


0:	learn: 0.6483510	total: 13.6ms	remaining: 13.6s
999:	learn: 0.1273823	total: 26.8s	remaining: 0us


[I 2024-03-13 18:54:29,378] Trial 1 finished with value: 0.760414914229804 and parameters: {'undersampling': False, 'n_unique_to_cat_cols': 20, 'iterations': 1000, 'depth': 7, 'l2_leaf_reg': 5, 'border_count': 64, 'max_ctr_complexity': 0, 'bootstrap_type': 'Bayesian', 'bagging_temperature': 0.12783311144653642}. Best is trial 0 with value: 0.7621774084938122.


...

0:	learn: 0.6669582	total: 8.11ms	remaining: 12.2s
1000:	learn: 0.2870467	total: 7s	remaining: 3.49s
1499:	learn: 0.2792449	total: 10.6s	remaining: 0us


[I 2024-03-13 20:35:03,680] Trial 48 finished with value: 0.7616369083005139 and parameters: {'undersampling': True, 'n_unique_to_cat_cols': 0, 'iterations': 1500, 'depth': 5, 'l2_leaf_reg': 19, 'border_count': 64, 'max_ctr_complexity': 5, 'bootstrap_type': 'Bernoulli', 'subsample': 0.9646764399070219}. Best is trial 31 with value: 0.7629860683487327.


0:	learn: 0.6666321	total: 21.8ms	remaining: 10.9s
499:	learn: 0.3008943	total: 10.9s	remaining: 0us


[I 2024-03-13 20:35:24,149] Trial 49 finished with value: 0.7538583366876188 and parameters: {'undersampling': True, 'n_unique_to_cat_cols': 5, 'iterations': 500, 'depth': 4, 'l2_leaf_reg': 21, 'border_count': 64, 'max_ctr_complexity': 6, 'bootstrap_type': 'Bernoulli', 'subsample': 0.8051440267922123}. Best is trial 31 with value: 0.7629860683487327.


Number of finished trials: 50
Best trial:
  Value: 0.7629860683487327
  Params:
    undersampling: True
    n_unique_to_cat_cols: 15
    iterations: 1500
    depth: 7
    l2_leaf_reg: 21
    border_count: 64
    max_ctr_complexity: 6
    bootstrap_type: Bernoulli
    subsample: 0.9275798959802446


In [10]:
final_params = trial.params
final_params = {
    k: (round(v, 2) if isinstance(v, float) else v)
    for k, v in final_params.items()
}
final_params

{'undersampling': True,
 'n_unique_to_cat_cols': 15,
 'iterations': 1500,
 'depth': 7,
 'l2_leaf_reg': 21,
 'border_count': 64,
 'max_ctr_complexity': 6,
 'bootstrap_type': 'Bernoulli',
 'subsample': 0.93}

In [11]:
final_params = {**final_params, **default_params}
final_params

{'undersampling': True,
 'n_unique_to_cat_cols': 15,
 'iterations': 1500,
 'depth': 7,
 'l2_leaf_reg': 21,
 'border_count': 64,
 'max_ctr_complexity': 6,
 'bootstrap_type': 'Bernoulli',
 'subsample': 0.93,
 'random_state': 42,
 'allow_writing_files': False,
 'verbose': 1000,
 'task_type': 'GPU'}

In [9]:
final_params = {
    'undersampling': True,
    'n_unique_to_cat_cols': 15,
    'iterations': 1500,
    'depth': 7,
    'l2_leaf_reg': 21,
    'border_count': 64,
    'max_ctr_complexity': 6,
    'bootstrap_type': 'Bernoulli',
    'subsample': 0.93,
    'random_state': 42,
    'allow_writing_files': False,
    'verbose': 1000,
    'task_type': 'GPU',
}
final_params

{'undersampling': True,
 'n_unique_to_cat_cols': 15,
 'iterations': 1500,
 'depth': 7,
 'l2_leaf_reg': 21,
 'border_count': 64,
 'max_ctr_complexity': 6,
 'bootstrap_type': 'Bernoulli',
 'subsample': 0.93,
 'random_state': 42,
 'allow_writing_files': False,
 'verbose': 1000,
 'task_type': 'GPU'}

In [10]:
if final_params['undersampling']:
    Xy_train = data.group_by(
        'target'
    ).head(
        100_000
    ).to_pandas()
else:
    Xy_train = data.to_pandas()

X_train_sampled, y_train_sampled = Xy_train.drop(columns=['target']), Xy_train['target'].values

cat_cols = n_unique_vals_by_col.filter(
    pl.col('n_unique') <= final_params['n_unique_to_cat_cols']
)['column'].to_list()

train_pool = Pool(X_train_sampled, label=y_train_sampled, cat_features=cat_cols)

model = CatBoostClassifier(**{k: v for k, v in final_params.items() if k not in ['undersampling', 'n_unique_to_cat_cols']})
model.fit(train_pool)

0:	learn: 0.6724481	total: 5.88s	remaining: 2h 26m 59s
1000:	learn: 0.3394862	total: 1m 24s	remaining: 42s
1499:	learn: 0.3239409	total: 2m 3s	remaining: 0us


In [11]:
data_test = pl.read_parquet(DATA_PATH + 'data_test_cleaned.parquet')

In [12]:
X_test = data_test.drop('target').to_pandas()

y_pred = model.predict_proba(X_test)[:, 1]
y_pred_binary = (y_pred > round(y_train.mean(), 3)).astype(int)

In [13]:
submission = pl.read_parquet(DATA_PATH + 'test_sber.parquet').select('id').to_pandas()

submission['target_prob'] = y_pred
submission['target_bin'] = y_pred_binary

submission.to_csv('submission_2_top_n.csv', index=False)
submission.head(3)

,id,target_prob,target_bin
0,3,0.061472,1
1,4,0.084514,1
2,12,0.150282,1


## 2. Feature selection

In [14]:
def select_features(
    pool_train: Pool, pool_valid: Pool, final_params, features,
    algorithm: EFeaturesSelectionAlgorithm,
    num_features_to_select: int = 250,
    steps: int = 25,
):
    """
    taken and updated from:
    https://github.com/catboost/catboost/blob/master/catboost/
    tutorials/feature_selection/select_features_tutorial.ipynb
    """
    print('Algorithm:', algorithm)
    model = CatBoostClassifier(**final_params)

    summary = model.select_features(
        pool_train,
        eval_set=pool_valid,
        features_for_select=features,
        num_features_to_select=num_features_to_select,
        steps=steps,                                     # more steps - more accurate selection
        algorithm=algorithm,
        shap_calc_type=EShapCalcType.Regular,            # can be Approximate, Regular and Exact
        train_final_model=True,                          # to train model with selected features
        logging_level='Verbose',
        plot=True,
    )
    print('Selected features:', summary['selected_features_names'])
    print()
    print('Eliminated features:', summary['eliminated_features_names'])

    return summary

In [15]:
if final_params['undersampling']:
    X_train['y'] = y_train
    Xy_train = X_train.groupby('y').head(100_000)

    X_train_sampled, y_train_sampled = Xy_train.drop(columns=['y']), Xy_train['y'].values
else:
    X_train_sampled, y_train_sampled = X_train.copy(), y_train.copy()

cat_cols = n_unique_vals_by_col.filter(
    pl.col('n_unique') <= final_params['n_unique_to_cat_cols']
)['column'].to_list()

train_pool = Pool(X_train_sampled, label=y_train_sampled, cat_features=cat_cols)
valid_pool = Pool(X_valid, label=y_valid, cat_features=cat_cols)

summary = select_features(
    pool_train=train_pool, pool_valid=valid_pool, final_params={k: v for k, v in final_params.items() if k not in ['undersampling', 'n_unique_to_cat_cols']},
    features = X_valid.columns.to_list(), algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues, num_features_to_select=250, steps=25,
)

Algorithm: EFeaturesSelectionAlgorithm.RecursiveByShapValues


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Step #1 out of 25
0:	learn: 0.6691895	test: 0.6641792	best: 0.6641792 (0)	total: 86ms	remaining: 2m 8s
1000:	learn: 0.3000189	test: 0.1787809	best: 0.1787775 (999)	total: 1m 17s	remaining: 38.9s
1499:	learn: 0.2895844	test: 0.1781511	best: 0.1781442 (1490)	total: 1m 56s	remaining: 0us
bestTest = 0.1781442179
bestIteration = 1490
Shrink model to first 1491 iterations.
Feature #605 eliminated
Feature #594 eliminated
Feature #249 eliminated
Feature #263 eliminated
Feature #595 eliminated
Feature #582 eliminated
Feature #318 eliminated
Feature #20 eliminated
Feature #608 eliminated
Feature #490 eliminated
Feature #587 eliminated
Feature #399 eliminated
Feature #604 eliminated
Feature #160 eliminated
Feature #467 eliminated
Feature #616 eliminated
Feature #161 eliminated
Feature #512 eliminated
Feature #242 eliminated
Feature #601 eliminated
Feature #65 eliminated
Feature #353 eliminated
Feature #589 eliminated
Feature #199 eliminated
Step #2 out of 25
0:	learn: 0.6696105	test: 0.6645759	be

Selected features: ['feature36', 'feature38', 'feature41', 'feature44', 'feature45', 'feature48', 'feature49', 'feature52', 'feature53', 'feature56', 'feature57', 'feature58', 'feature59', 'feature60', 'feature61', 'feature64', 'feature65', 'feature70', 'feature71', 'feature72', 'feature74', 'feature75', 'feature76', 'feature78', 'feature79', 'feature80', 'feature81', 'feature82', 'feature83', 'feature84', 'feature85', 'feature86', 'feature87', 'feature92', 'feature95', 'feature97', 'feature100', 'feature102', 'feature104', 'feature106', 'feature111', 'feature112', 'feature113', 'feature114', 'feature115', 'feature117', 'feature123', 'feature125', 'feature127', 'feature128', 'feature129', 'feature133', 'feature140', 'feature146', 'feature150', 'feature153', 'feature161', 'feature163', 'feature164', 'feature166', 'feature167', 'feature168', 'feature169', 'feature171', 'feature186', 'feature196', 'feature197', 'feature202', 'feature203', 'feature205', 'feature214', 'feature215', 'feature

In [16]:
if final_params['undersampling']:
    Xy_train = data.group_by(
        'target'
    ).head(
        100_000
    ).to_pandas()
else:
    Xy_train = data.to_pandas()

X_train_sampled, y_train_sampled = Xy_train.drop(columns=['target']).loc[:, summary['selected_features_names']], Xy_train['target'].values

cat_cols = n_unique_vals_by_col.filter(
    pl.col('n_unique') <= final_params['n_unique_to_cat_cols']
)['column'].to_list()

cat_cols = [col for col in cat_cols if col in summary['selected_features_names']]

train_pool = Pool(X_train_sampled, label=y_train_sampled, cat_features=cat_cols)

model = CatBoostClassifier(**{k: v for k, v in final_params.items() if k not in ['undersampling', 'n_unique_to_cat_cols']})
model.fit(train_pool)

0:	learn: 0.6757779	total: 26.8ms	remaining: 40.2s
1000:	learn: 0.3916732	total: 25.8s	remaining: 12.8s
1499:	learn: 0.3799258	total: 38.7s	remaining: 0us


In [19]:
data_test = pl.read_parquet(DATA_PATH + 'data_test_cleaned.parquet')

In [20]:
X_test = data_test.drop('target').to_pandas()

X_test = X_test.loc[:, summary['selected_features_names']]

y_pred = model.predict_proba(X_test)[:, 1]
y_pred_binary = (y_pred > round(y_train.mean(), 3)).astype(int)

In [21]:
submission = pl.read_parquet(DATA_PATH + 'test_sber.parquet').select('id').to_pandas()

submission['target_prob'] = y_pred
submission['target_bin'] = y_pred_binary

submission.to_csv('submission_2_top_250.csv', index=False)
submission.head(3)

,id,target_prob,target_bin
0,3,0.141666,1
1,4,0.107053,1
2,12,0.102647,1
